In [4]:
import pandas as pd

In [5]:
# Adjusting the generation process to create a dataset with exactly 2000 entries
# Define all Indian states and fish types again for clarity in this standalone code block

import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(42)

# Fish types and Indian states defined previously
fish_types = [
    "Gilt Head Bream", "Red Sea Bream", "Sea Bass", "Red Mullet", "Horse Mackerel",
    "Black Sea Sprat", "Striped Red Mullet", "Trout", "Shrimp"
]

all_states = [
    "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", "Goa",
    "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala",
    "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland",
    "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura",
    "Uttar Pradesh", "Uttarakhand", "West Bengal", "Andaman and Nicobar Islands",
    "Chandigarh", "Dadra and Nagar Haveli and Daman and Diu", "Delhi", "Lakshadweep",
    "Puducherry"
]

# Calculate how many entries per fish type per state to reach 2000 entries
entries_per_state_per_fish = 2000 // (len(all_states) * len(fish_types))

# Generate the dataset
data = []
for state in all_states:
    for fish in fish_types:
        for _ in range(entries_per_state_per_fish):
            average_consumption = np.random.uniform(5, 20)  # in tons
            preferred_size = np.random.uniform(10, 40)  # in cm
            price_range_low = np.random.uniform(200, 500)  # in INR per kg
            price_range_high = price_range_low + np.random.uniform(50, 150)  # in INR per kg
            seasonal_availability = np.random.choice(['Year-round', 'Seasonal'])
            
            data.append({
                "State": state,
                "Fish Type": fish,
                "Average Consumption (tons)": round(average_consumption, 2),
                "Preferred Size (cm)": round(preferred_size, 2),
                "Price Range (INR per kg)": f"{round(price_range_low, 2)} - {round(price_range_high, 2)}",
                "Seasonal Availability": seasonal_availability
            })

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
csv_path = 'fish_demand_dataset_india_2000.csv'
df.to_csv(csv_path, index=False)

csv_path


'fish_demand_dataset_india_2000.csv'

In [4]:
import pandas as pd
import numpy as np
df=pd.read_csv("fish_demand_dataset_india_2000.csv")

In [8]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import numpy as np
import pandas as pd

def fish_recommendations_for_state(state, df):
    # Create a user-item matrix where rows represent states, columns represent fish types, and values represent consumption
    user_item_matrix = df.pivot_table(index='State', columns='Fish Type', values='Average Consumption (tons)', fill_value=0)

    # Convert user-item matrix to sparse matrix
    sparse_matrix = csr_matrix(user_item_matrix.values)

    # Perform SVD
    k = min(user_item_matrix.shape) - 1  # Number of latent factors (choose one less than the smaller dimension)
    U, sigma, Vt = svds(sparse_matrix, k=k)

    # Reconstruct the matrix
    predicted_consumption = np.dot(np.dot(U, np.diag(sigma)), Vt)

    # Convert the predicted consumption to DataFrame
    predicted_df = pd.DataFrame(predicted_consumption, columns=user_item_matrix.columns, index=user_item_matrix.index)

    # Get predicted consumption for the given state
    state_consumption = predicted_df.loc[state]

    # Sort fish types by predicted consumption and recommend the top ones
    recommended_fish = state_consumption.sort_values(ascending=False).head(3)
    
    return recommended_fish.index.tolist()

# Example usage:
state = "Kerala"  # You can change this to any state
recommendations = fish_recommendations_for_state(state, df)
print(f"Recommendations for {state}:")
print(recommendations)


Recommendations for Kerala:
['Horse Mackerel', 'Sea Bass', 'Striped Red Mullet']


In [6]:
import pandas as pd

def fish_demand_info(fish_name, df):
    # Filter the DataFrame for the given fish name
    fish_data = df[df['Fish Type'] == fish_name]
    
    # Check if data for the fish exists
    if fish_data.empty:
        return f"No information available for {fish_name}"
    
    # Calculate statistics
    avg_consumption = fish_data['Average Consumption (tons)'].mean()
    preferred_size = fish_data['Preferred Size (cm)'].mean()
    price_low = fish_data['Price Range (INR per kg)'].apply(lambda x: float(x.split(' - ')[0])).mean()
    price_high = fish_data['Price Range (INR per kg)'].apply(lambda x: float(x.split(' - ')[1])).mean()
    seasonal_availability = fish_data['Seasonal Availability'].mode().iloc[0]
    
    # Construct info string
    info = f"Demand info for {fish_name}:\n"
    info += f"Average consumption: {avg_consumption:.2f} tons\n"
    info += f"Preferred size: {preferred_size:.2f} cm\n"
    info += f"Price range: INR {price_low:.2f} - {price_high:.2f} per kg\n"
    info += f"Seasonal availability: {seasonal_availability}"
    
    return info

# Example usage:
fish_name = "Sea Bass"  # You can change this to any fish name
demand_info = fish_demand_info(fish_name, df)
print(demand_info)


Demand info for Sea Bass:
Average consumption: 12.71 tons
Preferred size: 26.29 cm
Price range: INR 342.75 - 443.39 per kg
Seasonal availability: Seasonal
